## 빅데이터 실습

### 스타벅스매장 입지 분석

#### 데이터 수집

##### 데이터 전처리 마무리

##### 스타벅스 분석 시각화

In [ ]:
# 판다스
import pandas as pd
# 시각화 라이브러리
import folium
import json

In [ ]:
# 스타벅스 서울시 매장정보 로드
dfStarbucksSeoul = pd.read_excel('./data/starbucks_seoul_list.xlsx')

In [ ]:
# 주소를 ' '로 자른다음(배열) 1번쨰 인덱스 값 구하기
guName = []
for address in dfStarbucksSeoul['주소']:
    addr = address.split(' ')
    guName.append(addr[1])

dfStarbucksSeoul['구']= guName

In [ ]:
dfStarbucksSeoul.to_excel('./data/최종_서울_스타벅스매장list.xlsx',index=False)

In [ ]:
# 스타벅스 분석 데이터 만들기
dfSeoulGuList = pd.read_excel('./data/서울_위경도.xlsx')
dfSeoulGuList.tail()
dfSeoulGuLast = dfSeoulGuList[['do','city','latitude','longitude']]
dfSeoulGuLast.to_excel('./data/최종_서울_구리스트.xlsx',index=False)

In [ ]:
# 각 칼럼의 영어이름은 한글 이름으로 변경
dfSeoulGuLast.columns = ['시','구','위도','경도']
dfSeoulGuLast.tail()
dfSeoulGuLast.to_excel('./data/최종_서울_구리스트.xlsx',index=False)

In [ ]:
# 스타벅스 매장에서 구로 그룹화, 매장명수 카운팅(pivot_table)
starbucksSeoulCount = dfStarbucksSeoul.pivot_table(values='매장명',
                             index='구',aggfunc='count').rename(columns={'매장명':'매장수'})
starbucksSeoulCount.tail()

In [ ]:
# 서울구 리스트에 스타벅스구별 매장수를 병합
lastSbSeoulCount = pd.merge(left=dfSeoulGuLast,right=starbucksSeoulCount,how='left',on='구')

In [ ]:
#서울시 구별 인구통계로드
dfSeoulGuPop = pd.read_excel('./data/수정_서울_구별인구.xlsx')
dfSeoulGuPop.tail()

In [ ]:
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount,right=dfSeoulGuPop[['구','성별소계']],how='left',on='구')
lastSbSeoulCount.tail()

In [ ]:
# 사업체 정보에 들어올 종사자수 컬럼이름이 '성별소계'로 동일, 컬럼이름 변경
lastSbSeoulCount.columns = ['시','구','위도','경도','스타벅스매장수','주민등록인구']

In [ ]:
# 사업체 정보 로드
dfSeoulGuBiz = pd.read_excel('./data/수정_서울_구별_사업체현황.xlsx')

In [ ]:
# 사업체정보에서 컬럼이름 부터 변경
dfSeoulGuBiz = dfSeoulGuBiz[['구','소계','성별소계']]
dfSeoulGuBiz.columns=['구','사업체수','종사가주']

In [ ]:
# 사업체정보를 스타벅스 최종데이터 병합
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount,right=dfSeoulGuBiz,how='left',on='구')

In [ ]:
lastSbSeoulCount.to_excel('./data/최종_서울_스타벅스매장_데이터.xlsx',index=False)

### 시각화

In [ ]:
# 서울시 스타벅스 매장위치 데이터 로드
dfStarbucksSeoul = pd.read_excel('./data/최종_서울_스타벅스매장list.xlsx')

In [ ]:
# 데이터 불러오기
dfStarbucksSeoulCount = pd.read_excel('./data/최종_서울_스타벅스매장_데이터.xlsx')

In [ ]:
dfStarbucksSeoul['매장타입'].velue_counts()

In [ ]:
# 서울시 지도 생성
seoulLoc=[37.550800, 126.989857]
sbMap = folium.Map(location=seoulLoc,zoom_start=11)

# 각 매장 위치를 지도에 마커로 표시
for idx in dfStarbucksSeoul.index:
    lat = dfStarbucksSeoul.loc[idx,'위도']
    lng = dfStarbucksSeoul.loc[idx,'경도']
    
    # 매장타입별 색상
    
    
    # 서클마커
    folium.CircleMarker(location=[lat,lng],fill=True,fill_color='green',fill_opacity=1,
                        color = 'yellow',weight=1,radius=3).add_to(sbMap)

sbMap